Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [10]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [7]:
train['description'][0]

'\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

### Define Pipeline Components

In [76]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=5)
rfc = RandomForestClassifier()

pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('rfc', rfc)
                ])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [77]:
params = {
    'vect__max_df': (0.75, 1.0),
    'rfc__max_depth':(35, 40, 45),
    'rfc__n_estimators':(150, 200),
    'rfc__min_samples_leaf':(1,2)
}

print("Starting Grid Search for these parameters:")

for key in params:
    print('\n', key + ":")
    for value in params[key]:
        print('\t', value)
print("\n")
    
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

print("Completed grid search")

Starting Grid Search for these parameters:

 vect__max_df:
	 0.75
	 1.0

 rfc__max_depth:
	 35
	 40
	 45

 rfc__n_estimators:
	 150
	 200

 rfc__min_samples_leaf:
	 1
	 2


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   40.6s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:  2.0min finished


Completed grid search


In [84]:
print("Best score:\t", grid_search.best_score_)
print("\nBest Parameters:\n", grid_search.best_params_)

Best score:	 0.7374615819699358

Best Parameters:
 {'rfc__max_depth': 35, 'rfc__min_samples_leaf': 1, 'rfc__n_estimators': 200, 'vect__max_df': 1.0}


### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [26]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [27]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [29]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [34]:
submission['ratingCategory'].dtype

dtype('int64')

In [32]:
subNumber = 0

In [33]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [85]:
# get our parameters from the first grid search
grid_search.best_params_

{'rfc__max_depth': 35,
 'rfc__min_samples_leaf': 1,
 'rfc__n_estimators': 200,
 'vect__max_df': 1.0}

In [86]:
from sklearn.decomposition import TruncatedSVD

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=5)
rfc = RandomForestClassifier(max_depth=45, min_samples_leaf=2, n_estimators=200)
svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)


lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('rfc', rfc)])

In [87]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[0.75, .9, 1.0],
}

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [88]:
print("Starting Grid Search for these parameters:")

for key in params:
    print('\n', key + ":")
    for value in params[key]:
        print('\t', value)
print("\n")
    
grid_search2 = GridSearchCV(pipe, params, cv=5, n_jobs=4, verbose=1)
grid_search2.fit(train['description'], train['ratingCategory'])

print("Completed grid search")

Starting Grid Search for these parameters:

 vect__max_df:
	 0.75
	 1.0

 rfc__max_depth:
	 35
	 40
	 45

 rfc__n_estimators:
	 150
	 200

 rfc__min_samples_leaf:
	 1
	 2


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:  2.6min finished


Completed grid search


In [89]:
print("Best score:\t", grid_search2.best_score_)
print("\nBest Parameters:\n", grid_search2.best_params_)

Best score:	 0.7235179693134581

Best Parameters:
 {'lsi__svd__n_components': 100, 'lsi__vect__max_df': 1.0}


### Make a Submission File

In [90]:
# Predictions on test sample
pred = grid_search2.predict(test['description'])

In [91]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [92]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [93]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [1]:
# Apply to your Dataset

# we'll use grid search for now
"""
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}
"""

In [79]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")


In [80]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [83]:
X = get_word_vectors(train['description'])
X_test = get_word_vectors(test['description'])

len(X) == len(train['description'])

True

In [105]:
rfc = RandomForestClassifier(max_depth=35, min_samples_leaf=5, n_estimators=200)

rfc.fit(X, train['ratingCategory'])

In [101]:
rfc.score(X, train['ratingCategory'])

0.9471494984095914

In [102]:
rfc.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [103]:
test['ratingCategory'] = rfc.predict(X_test)

In [107]:
test[['id', 'ratingCategory']].to_csv('testSolutionSubmission.csv', header=True, index=False)

### Try to improve the embeddings model

In [238]:
# let's see if we can improve results with a grid search


rfc = RandomForestClassifier(class_weight = "balanced" )
params = {
    'max_depth':(40, 50, 60),
    'n_estimators':(100, 150, 200),
    'min_samples_leaf':(1,2,3,5,10)
}


In [ ]:
from sklearn.metrics import f1_score
import sklearn

# use to see list of all accepted scoreing metrics
#sorted(sklearn.metrics.SCORERS.keys())

# Use to set a scoring metric.
# Submission scores were lower for models that used the f1 score for tuning
#scoring_metric = "f1_micro"


In [227]:

print("Starting Grid Search for these parameters:")

for key in params:
    print('\n', key + ":")
    for value in params[key]:
        print('\t', value)
print("\n")
    
grid_search3 = GridSearchCV(rfc, 
                            params, 
                            cv=5, 
                            n_jobs=4, 
                            verbose=1
                            #, scoring=scoring_metric
                           )
grid_search3.fit(X, train['ratingCategory'])

print("Completed grid search")

Starting Grid Search for these parameters:

 max_depth:
	 40
	 50
	 60

 n_estimators:
	 100
	 150
	 200

 min_samples_leaf:
	 1
	 2
	 3
	 5
	 10


Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  6.0min
[Parallel(n_jobs=4)]: Done 225 out of 225 | elapsed:  7.0min finished


Completed grid search


In [237]:
print("Best score:\t", grid_search3.best_score_)
print("\nBest Parameters:\n", grid_search3.best_params_)

Best score:	 0.7313461198911877

Best Parameters:
 {'max_depth': 40, 'min_samples_leaf': 3, 'n_estimators': 200}


In [241]:
# Table of our results for the Randomized Search
results = pd.DataFrame(grid_search3.cv_results_).sort_values(by='rank_test_score')
results[["mean_test_score", "param_max_depth", "param_min_samples_leaf", "param_n_estimators"]].head(10)

,mean_test_score,param_max_depth,param_min_samples_leaf,param_n_estimators
8,0.731346,40,3,200
23,0.731101,50,3,200
21,0.729387,50,3,100
11,0.729143,40,5,200
7,0.728165,40,3,150
26,0.727924,50,5,200
40,0.727190,60,5,150
38,0.727188,60,3,200
41,0.726699,60,5,200
33,0.726449,60,2,100


### Make a Submission File

In [233]:
# Predictions on test sample
pred = grid_search3.predict(X_test)

In [230]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [232]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False
subNumber += 1

### Let's try Randomized Search

In [197]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

rfc = RandomForestClassifier()
params = {
    'max_depth':[40, 45, 50, 55],
    'n_estimators':randint(160, 220),
    'min_samples_leaf': randint(1,10)
}


In [201]:
print("Starting randomized search for these parameters:")

for key in params:
    print('\n' + key + ":")
    if type(params[key]) is list:
        for value in params[key]:
            print('\t', value)
    else:
        print(params[key])

print("\n")

random_search = RandomizedSearchCV(rfc, 
                            params, 
                            cv=5, 
                            n_jobs=4, 
                            verbose=1
                            #, scoring=scoring_metric
                           )
random_search.fit(X, train['ratingCategory'])

print("Completed randomized search")

Starting randomized search for these parameters:

max_depth:
	 40
	 45
	 50
	 55

n_estimators:

min_samples_leaf:


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  2.1min finished


Completed randomized search


In [202]:
print("Best score:\t", random_search.best_score_.round(4))
print("\nBest Parameters:\n", random_search.best_params_)

Best score:	 0.7375

Best Parameters:
 {'max_depth': 55, 'min_samples_leaf': 7, 'n_estimators': 206}


In [203]:
# Table of our results for the Randomized Search
results = pd.DataFrame(random_search.cv_results_).sort_values(by='rank_test_score')
results[["mean_test_score", "param_max_depth", "param_min_samples_leaf", "param_n_estimators"]]

,mean_test_score,param_max_depth,param_min_samples_leaf,param_n_estimators
3,0.737459,55,7,206
1,0.734525,45,2,174
4,0.734283,40,1,195
2,0.734283,40,2,188
6,0.734282,40,2,190
9,0.734278,55,3,191
5,0.733302,40,3,211
7,0.732324,45,4,216
0,0.731097,40,4,177
8,0.729874,55,5,192


In [205]:
# Predictions on test sample
pred = random_search.predict(X_test)

submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

### Let's try Gradient Boost

In [211]:
from sklearn.ensemble import GradientBoostingClassifier

"""
NOTE TO SELF:

Might want to consider XGBoost. Would have to install in the conda env, but might give better results.
"""

In [254]:
from sklearn.metrics import f1_score

# Use to set a scoring metric.

scoring_metric = "f1_micro"

gbc = GradientBoostingClassifier()
params = {
    'max_depth':[5,10],
    'n_estimators':[70, 100, 130],
    'min_samples_leaf': randint(3,7)
}


In [255]:


print("Starting randomized search for these parameters:")

for key in params:
    print('\n' + key + ":")
    if type(params[key]) is list:
        for value in params[key]:
            print('\t', value)
    else:
        print(params[key])

print("\n")

random_search2 = RandomizedSearchCV(gbc, 
                            params, 
                            cv=5, 
                            n_jobs=4, 
                            verbose=1
                            , scoring=scoring_metric
                           )
random_search2.fit(X, train['ratingCategory'])

print("Completed randomized search")

Starting randomized search for these parameters:

max_depth:
	 5
	 10

n_estimators:
	 70
	 100
	 130

min_samples_leaf:


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 53.2min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed: 60.4min finished


Completed randomized search


In [258]:
print("Best score:\t", random_search2.best_score_.round(4))
print("\nBest Parameters:\n", random_search2.best_params_)

Best score:	 0.7416

Best Parameters:
 {'max_depth': 5, 'min_samples_leaf': 4, 'n_estimators': 130}


In [259]:
# Table of our results for the Randomized Search
results = pd.DataFrame(random_search2.cv_results_).sort_values(by='rank_test_score')
results[["mean_test_score", "param_max_depth", "param_min_samples_leaf", "param_n_estimators"]]

,mean_test_score,param_max_depth,param_min_samples_leaf,param_n_estimators
7,0.741622,5,4,130
0,0.738930,5,3,70
5,0.737705,10,3,130
3,0.737462,5,6,130
9,0.737460,5,5,130
8,0.736727,5,3,130
2,0.734771,5,3,70
4,0.734524,10,6,70
6,0.734524,10,3,130
1,0.730608,5,3,70


In [260]:
# Predictions on test sample
pred = random_search2.predict(X_test)

submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    Sentimant Analysis refers to analyzing a text and determining underlying sentiment or emotion. The analysis can be basic (i.e. is the sentiment positive, negative, or neutral>).
    Or it can be more advanced (i.e. which emotional state is reflected in the text?)
    
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    
    
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?